<div style="background:#222222; color:#ffffff; padding:20px">
<h1 align="center">Guided ML With The Iris Dataset</h1>

<h2 align="center" tyle="color:#01ff84" >Learning type | Activity type | Objective |</h2>
<h2 align="center">| Supervised | Multiclass classification | Identify a flower's class |</h2>


<div>

Contents:

1. Loading the data
2. Setting up supervised learning problem (selecting features)
3. Creating a first model
    - Creating train and test datasets
    - Normalizing train and test
    - Fitting and predicting
4. Evaluate the frist model predictions
5. Crossvalidation of the model
6. Creating an end to end ML pipeline
    - Train/Test Split
    - Normalize
    - Crossvalidations
    - Model
    - fitting and predicting

## Instructions with NBGrader removed

Complete the cells beginning with `# YOUR CODE HERE` and run the subsequent cells to check your code.


Contents:
1. Loading the data
2. Setting up supervised learning problem (selecting features)
3. Creating a first model
    - Creating train and test datasets
    - Normalizing train and test
    - Fitting and predicting
4. Evaluate the frist model predictions
5. Crossvalidation of the model
6. Creating an end to end ML pipeline
    - Train/Test Split
    - Normalize
    - Crossvalidations
    - Model
    - fitting and predicting

## About the dataset

[Iris](https://archive.ics.uci.edu/ml/datasets/iris) is a well-known multiclass dataset. It contains 3 classes of flowers with 50 examples each. There are a total of 4 features for each flower.

![](./classic-datasets/images/Iris-versicolor-21_1.jpg)

## Package setups

1. Run the following two cells to initalize the required libraries. 

In [1]:
#to debug package errors
import sys
sys.path
sys.executable


'C:\\Users\\Abubakr\\Anaconda3\\envs\\ai\\python.exe'

In [2]:
# Import needed packages
# You may add or remove packages should you need them
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Display plots inline and change plot resolution to retina
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Set Seaborn aesthetic parameters to defaults
sns.set()

## Step 1: Loading the data

1. Load the iris dataset using ```datasets.load_iris()```
2. Investigate the data structure with ```.keys()```
3. Construct a dataframe from the dataset
4. Create a 'target' and a 'class' column that contains the target names and values
5. Display a random sample of the dataframe 

In [3]:
def load_data():
    #load the dataset
    #return the dataset
    # YOUR CODE HERE
    data = datasets.load_iris()
    return data

In [4]:
assert load_data()['data'].shape == (150,4)

In [5]:
dataset = load_data()
load_data().feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
def dataset_to_pandas():
    #put the dataset into a pandas DF using the feature names as columnsç
    #rename the column name so the dont include the '(cm)'
    #add 2 columns one with the target and another with the target_names
    
    # YOUR CODE HERE
    df = pd.DataFrame(dataset.data)
    df.columns = dataset.feature_names
    df.rename(columns={'sepal length (cm)':'sepal length',
                       'sepal width (cm)':'sepal width',
                       'petal length (cm)':'petal length',
                       'petal width (cm)':'petal width'},inplace=True)
    df['target'] = dataset.target
    df['class'] = dataset.target_names[dataset.target]
    return df

In [7]:
df = dataset_to_pandas()
assert df['sepal length'].shape == (150,)
assert df['sepal width'].shape == (150,)
assert df['petal length'].shape == (150,)
assert df['petal width'].shape == (150,)
assert df['target'].shape == (150,)
assert df['class'].shape == (150,)


### Question
Find the X and y values we're looking for. Notice that y is categorical and thus, we could **one-hot encode it** if we are looking at **class** or we can just pick **target**. In order to one hot encode we have  to re-shape `y` it using the **.get_dummies** function. 

## For the purpose of this exercise, do not use hot encoding, go only for target but think about if you have to drop it somewhere or not...

In [8]:
df_iris = dataset_to_pandas()


In [9]:
df_iris

,sepal length,sepal width,petal length,petal width,target,class
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


In [10]:
# virginica close to versicolor BAD
#0 - setosa
#1 - versicolor
#2 - virginica 

# Same distance between all the classes GOOD
#[1,0,0] - setosa
#[0,1,0] - versicolor
#[0,0,1] - virginica 


def ohe():
    # YOUR CODE HERE
    hot_enc = pd.get_dummies(df_iris['class'], prefix='data')
    dataframe=pd.concat([df_iris.drop('class', axis=1),hot_enc],axis=1)
    return dataframe

In [11]:
ohe_data = ohe()

assert ohe_data.shape == (150,8)

## Step 2: Setting up supervised learning problem (selecting features)

Feature selection is an essential step in improving a model's perfromance. In the first version of the model we will use the **'sepal length'** and **'sepal width'** as predicting features. Later we will see the effect of adding additional features.

1. Assign the values of the 'target' to Y as a numpy array
2. Assign the remaining feature values to X as a numpy array
3. Check the shape of X and Y. Check the first few values.
    - Can we confirm our X and Y are created correctly?

In [12]:
def target_to_numpy():
    # YOUR CODE HERE
    Y = np.array(ohe_data['target'])
    return Y
 
def data_to_numpy():
    # YOUR CODE HERE
    X = np.array(ohe_data[['sepal length','sepal width']])
    return X

In [13]:
Y = target_to_numpy()
X = data_to_numpy()
assert isinstance(Y, np.ndarray)
assert isinstance(X, np.ndarray)
assert X.shape == (150,2)

In [14]:
#your code here
X = df_iris[['sepal length', 'sepal width']].values
print(X.shape)
X[:5]

(150, 2)


array([[5.1, 3.5],
       [4.9, 3. ],
       [4.7, 3.2],
       [4.6, 3.1],
       [5. , 3.6]])

## Step 3: Creating the first model

In lecture we learned about creating a train and test datasets, normalizing, and fitting a model. In this step we will see how to build a simple version of this.

We have to be careful when constructing our train and test datasets. First, when we create train and test datasets we have to be careful that we always have the same datapoints in each set. Otherwise our results won't be reproduceable or we might introduce a bias into our model.

We also need to be attentive to when we normalize the data. What would be the effect of normalizing the data (i.e. with StandardScaler to a range between 0 - 1) before we create our train and test sets? Effectively we would use information in the test set to structure the values in the training set and vice versa. Therefore normalizing train and test independently is the preferred method.

1. Create X_train, X_test, Y_train, Y_test using ```train_test_split()``` with an 80/20 train/test split. Look in the SKLearn documentation to understand how the function works.
    - Inspect the first few rows of X_train.
    - Run the cell a few times. Do the first few rows change?
    - What option can we use in ```train_test_split()``` to stop this from happening?
2. Normalize the train and test datasets with ```StandardScaler```
    - We can fit the transform with ```.fit()``` and ```.transform()``` to apply it. Look in the documentation for an esample of how to do this.
    - Does it make sense to normalize Y_train and Y_test?
3. Initalize a ```LogisticRegression()``` model and use the ```.fit()``` method to initalize the first model.
    - We will pass the X_train and Y_train variables to the ```.fit()``` method.
    - Once the model is fit, use the ```.predict()``` with the X_test and save the output as predictions.

In [15]:
#split train and test data 80/20 use random_state=0, this is the state used for when the asserts were written
#your code here
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
# YOUR CODE HERE


print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(120, 2)
(120,)
(30, 2)
(30,)


In [16]:
assert X_train.shape == (120,2)
assert Y_train.shape == (120,)
assert X_test.shape  == (30,2)
assert Y_test.shape  == (30,)

In [17]:
#normalize the dataset
#your code here
scaler = StandardScaler()
#create and fit the scaler object on the training data
# YOUR CODE HERE
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train[:5]

array([[ 0.61303014,  0.10850105],
       [-0.56776627, -0.12400121],
       [-0.80392556,  1.03851009],
       [ 0.25879121, -0.12400121],
       [ 0.61303014, -0.58900572]])

In [18]:
assert np.amin(X_train) >= -2.5
assert np.amax(X_train) <= 3.2
assert np.amin(X_test) >= -2
assert np.amin(X_test) <= 2.75

In [19]:
#initalize and fit with Logistic Regression
prediction = 0
model = LogisticRegression()
model.fit(X_train,Y_train)
#your code here

#initalize the logistic regressor
#make predictions
# YOUR CODE HERE
predictions= model.predict(X_test)
predictions



array([1, 1, 0, 2, 0, 2, 0, 2, 2, 1, 1, 2, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 2, 0, 0, 2, 1, 0])

In [20]:
assert predictions.shape == (30,)

## Step 4: Evaluate the frist model's predictions

We will learn more about how to evaluate the performance of a classifier in later lessons. For now we will use % accuracy as our metric. It is important to know that this metric only helps us understand the specific performance of our model and not, for example, where we can improve it, or where it already perfoms well.

1. Use ```.score()``` to evaluate the performance of our first model.

In [21]:
score = 0
#evaluating the performace of our first model
#your code here
# YOUR CODE HERE
score = accuracy_score(Y_test,predictions)
score

0.7333333333333333

In [22]:
assert score >=0.7

In [23]:
assert score >=0.72

In [24]:
assert score >=0.73

## Step 5: Crossvalidation of the model
Our first model achived ~90% accruacy. This is quite good. How do we know it is reproducable? If we run the model again and our performance is 85% which is correct? And what about improving our model? Can you think of one thing we can do to potentially improve the model?

#### Crossvalidation
Corssvalidation is when we create multiple X and Y datasets. On each dataset we train and fit the model. We then average the results and return a 'crossvalidated' accruacy.

1. Initalize a new version of the model you trained above with the same paramters.
2. Use ```cross_validate()``` to run the model with 5 crossvalidation folds. 

In [25]:
#model with cross validation
#your code here
from sklearn import svm
#cross validate the training set
clf_cv = 0
CV = 0
# YOUR CODE HERE
clf_cv = svm.SVC(kernel='rbf', C=30, random_state=0)
cv = cross_validate(clf_cv,X_train,Y_train,cv=5)

def print_scores(cv):
    #print out cross validation scores
    [print('Crossvalidation fold: {}  Accruacy: {}'.format(n, score)) for n, score in enumerate(cv['test_score'])]
    #print out the mean of the cross validation
    print('Mean train cross validation score {}'.format(cv['test_score'].mean()))
    
print_scores(cv)

Crossvalidation fold: 0  Accruacy: 0.875
Crossvalidation fold: 1  Accruacy: 0.75
Crossvalidation fold: 2  Accruacy: 0.8333333333333334
Crossvalidation fold: 3  Accruacy: 0.75
Crossvalidation fold: 4  Accruacy: 0.7916666666666666
Mean train cross validation score 0.8


In [26]:
assert len(cv['test_score']) == 5
assert max(cv['test_score']) >= 0.85
assert min(cv['test_score']) >= 0.69
assert cv['test_score'].mean() >= 0.77

## Step 6: Creating an end to end ML pipeline
Congraulations you've trained, crossvalidated, predicted, and evaluated your frist classifier. Now that you understand the basic steps we will look at a way to combine all these steps together.

Before we go further think about what you would have to do if you wanted to change the model. Intalize a new model, change the vairables, redo the cross validation...etc. Seems like a lot. And when we have to change lots of code it is easy to make mistakes. And what if you wanted to try many models and see which one performed best? Or try changing many different features? How could you do it without writing each one out as we have?

The solution is to use SKLearn's pipeline class. A pipeline is an object that will execute the various steps in the machine learning process. We can choose what elements we want in the pipeline and those that we do not. Once setup, we can rapidly change models, or input data and have it return our results in an ordered way.


1. Initalize a scaler and a classifer object like we did previously.
2. Use the ```make_pipeline()``` function to construct a transofmraiton pipeline for the scaler and the classifier
3. Input the pipeline object to the cross_validator and evaluate with 5 folds.
4. Print out your results (hint: make a function for repetitve tasks like printing)

In [27]:

#define the scaler
#define the classifier
#make the pipeline
#run the cross validation
#print results
scaler = 0
classifier = 0
pipe = 0
scores = 0
# YOUR CODE HERE
scaler = StandardScaler()
classifier = LogisticRegression()
pipe = make_pipeline(scaler, classifier)
scores = cross_validate(pipe, X_train, Y_train, cv =5)


In [28]:
scores

{'fit_time': array([0.00996685, 0.01001859, 0.00797653, 0.00597501, 0.00597453]),
 'score_time': array([0.00099778, 0.00098872, 0.0010078 , 0.00099611, 0.0009954 ]),
 'test_score': array([0.91666667, 0.70833333, 0.83333333, 0.79166667, 0.79166667])}

In [29]:
assert type(pipe) == type(make_pipeline(scaler, classifier))
assert len(cv['test_score']) == 5
assert max(cv['test_score']) >= 0.83
assert min(cv['test_score']) >= 0.69
assert cv['test_score'].mean() >= 0.74

## Challenge Exercise

In this notebook we only used two features to predict the class of the flower. We also did not do any hypter parameter tuning. The challenge is to impove the prediction results. Some ideas we can try:
1. Add features to the input and run the cross validation pipeline
2. Investigate how to use ```GridSearchCV```, a powerful funtion that searches through hyperparmetrs and does cross validation.
    - Hint: Input the pipeline directly into GridSearchCV
3. Try a different models like RandomForest or SVM.

In [30]:
# YOUR CODE HERE
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20,30],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)

clf.fit(X_train,Y_train)
clf.cv_results_

{'mean_fit_time': array([0.0011951 , 0.00099506, 0.00119739, 0.00139813, 0.00199413,
        0.00160508, 0.00119166, 0.00139279]),
 'std_fit_time': array([4.02884752e-04, 4.60487448e-06, 4.00997381e-04, 4.87995539e-04,
        6.37393190e-04, 5.06173534e-04, 3.98636617e-04, 4.89704458e-04]),
 'mean_score_time': array([0.00059738, 0.00040145, 0.00099583, 0.00079722, 0.00099363,
        0.0007956 , 0.00080047, 0.00020237]),
 'std_score_time': array([4.87763547e-04, 4.91689091e-04, 1.79938910e-06, 3.98624696e-04,
        1.47176698e-05, 3.97802439e-04, 4.00254838e-04, 4.04739380e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_valu

In [31]:
# YOUR CODE HERE
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001195,0.000403,0.000597,0.000488,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.916667,0.708333,0.833333,0.791667,0.791667,0.808333,0.067700,1
1,0.000995,0.000005,0.000401,0.000492,1,linear,"{'C': 1, 'kernel': 'linear'}",0.833333,0.666667,0.791667,0.791667,0.791667,0.775000,0.056519,8
2,0.001197,0.000401,0.000996,0.000002,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.875000,0.750000,0.791667,0.791667,0.791667,0.800000,0.040825,5
3,0.001398,0.000488,0.000797,0.000399,10,linear,"{'C': 10, 'kernel': 'linear'}",0.875000,0.750000,0.833333,0.791667,0.791667,0.808333,0.042492,1
4,0.001994,0.000637,0.000994,0.000015,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.875000,0.750000,0.791667,0.791667,0.791667,0.800000,0.040825,5
5,0.001605,0.000506,0.000796,0.000398,20,linear,"{'C': 20, 'kernel': 'linear'}",0.875000,0.708333,0.833333,0.791667,0.750000,0.791667,0.058926,7
6,0.001192,0.000399,0.000800,0.000400,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.875000,0.750000,0.833333,0.750000,0.791667,0.800000,0.048591,3
7,0.001393,0.000490,0.000202,0.000405,30,linear,"{'C': 30, 'kernel': 'linear'}",0.875000,0.708333,0.833333,0.791667,0.791667,0.800000,0.055277,3


In [32]:
# YOUR CODE HERE
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.808333
1,1,linear,0.775000
2,10,rbf,0.800000
3,10,linear,0.808333
4,20,rbf,0.800000
5,20,linear,0.791667
6,30,rbf,0.800000
7,30,linear,0.800000


In [33]:
# YOUR CODE HERE
clf.best_score_

0.8083333333333333